In [4]:
# OBSERVATIONS:
# 1. As we knew before, our analysis shows that temperatures rise (get hotter) as we approach the equator. 
# 2. Temperatures also go down as we move towards the northern hemisphere (strong, negative correlation) and rise as we move towards the southern (strong, positive correlation).
# 3. Cloudiness, Humidity, and Wind Speed remain consistent with weak correlation across latitudes.

In [ ]:
# Dependencies & Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time 
from scipy.stats import linregress

# Import API Key
weather_api_key = "179a824b84e49bc6a67011fe2add49c8"

# Incorporated citipy to determine city based on latitude and longitude 
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [ ]:
# Generate Cities List 
# Lists for holding lat_lngs and cities 
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    # If the city is unique, then add it to our city list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count 
len(cities)

In [ ]:
# Perform API Calls 
# Starting URL for Weather Map API Key 
url = "http://api.openweathermap.org/data/2.5/weather?appid=179a824b84e49bc6a67011fe2add49c8&units=imperial"

In [ ]:
# Empty Lists 
name_of_city = []
country = []
date = []
lat = []
lng = []
temp = []
humidity = []
cloudiness = []
wind_speed = []

# Starting Points 
record = 1
set = 1

# Print 'Beginning Data Retrieval' Statement 
print(f"Beginning Data Retrieval")
print(f"-------------------------------")

# Create Loop through Cities in List
for i, city in enumerate(cities):
    # Group Cities in Sets of 50
    if (i % 50 == 0 and i >= 50):
        record = 0
        set += 1
    try:
        response = requests.get(f"{url}&q={city}").json()
        name_of_city.append(response["name"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        temp.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        wind_speed.append(response["wind"]["speed"])
        print_city = response["name"]
        
        print("Processing Record %s of Set %s | %s" % (record, set, print_city))
        
        # Increase counter
        record= record + 1
        # Timer
        time.sleep(1.00)
        
             
    # No record found, Go to Next 
    except:
        print("NO CITY FOUND. NEXT...")
    continue

# Data Retrieval Complete
print("----------------------------------")
print("Data Retrieval Complete           ")
print("----------------------------------")

In [ ]:
# Convert Raw Data to Dataframe 
weather_dictionary = {
    "City": name_of_city,
    "Lat": lat, 
    "Lng": lng,
    "Max Temp": temp, 
    "Humidity": humidity,
    "Cloudiness": cloudiness, 
    "Wind Speed": wind_speed, 
    "Country": country,
    "Date": date
}

# Data Frame
weather_frame = pd.DataFrame(weather_dictionary)

# Display Count 
weather_frame.count()

In [ ]:
# Save Data Frame to CSV
weather_frame.to_csv("output_csv/weather_frame.csv")
weather_frame.head()

In [ ]:
# Latitude vs. Temperature Plot
plt.scatter(weather_frame["Lat"], weather_frame["Max Temp"], facecolor="blue", edgecolor="black")
plt.title("City Latitude vs. Max Temperature (04/01/20)")
plt.xlabel("Laitude")
plt.ylabel("Max Temperature (F)")
plt.grid(linestyle='-',linewidth=1,alpha = 0.5)
plt.savefig("PNGs/City Latitude vs Max Temp.png")

In [ ]:
# Latitude vs. Humidity Plot
plt.scatter(weather_frame["Lat"], weather_frame["Humidity"], facecolor="blue", edgecolor="black")
plt.title("City Latitude vs. Humidity (04/01/20)")
plt.xlabel("Laitude")
plt.ylabel("Humidity (%)")
plt.grid(linestyle='-',linewidth=1,alpha = 0.5)
plt.savefig("PNGs/City Latitude vs Humidity.png")

In [ ]:
# Latitude vs. Cloudiness Plot
plt.scatter(weather_frame["Lat"], weather_frame["Cloudiness"], facecolor="blue", edgecolor="black")
plt.title("City Latitude vs. Cloudiness (04/01/20)")
plt.xlabel("Laitude")
plt.ylabel("Cloudiness (%)")
plt.grid(linestyle='-',linewidth=1,alpha = 0.5)
plt.savefig("PNGs/City Latitude vs Cloudiness.png")

In [ ]:
# Latitude vs. Wind Speed
plt.scatter(weather_frame["Lat"], weather_frame["Wind Speed"], facecolor="blue", edgecolor="black")
plt.title("City Latitude vs. Wind Speed (mph) (04/01/20)")
plt.xlabel("Laitude")
plt.ylabel("Wind Speed (%)")
plt.grid(linestyle='-',linewidth=1,alpha = 0.5)
plt.savefig("PNGs/City Latitude vs Wind Speed.png")

In [ ]:
# Linear Regression 
# Northern & Southern Data Frames 
northern = weather_frame.loc[weather_frame["Lat"] >= 0]
southern = weather_frame.loc[weather_frame["Lat"] < 0]

In [ ]:
# Linear Regression Function for Scatter Plot
import scipy.stats as st
def linear_agression(x,y):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    print(line_eq)
    plt.scatter(x, y)
    plt.plot(x,regress_values,"r-")
    print(f"The r-squared is : {rvalue**2}")

In [ ]:
# Northern Hemisphere - Max Temp vs. Latitude Linear Regression
equation = linear_agression(northern["Lat"], northern["Max Temp"])
plt.title("Northern Hemisphere - Max Temp vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")
plt.savefig("PNGs/Northern Hemisphere - Max Temp vs. Latitude Linear Regression.png")

In [ ]:
# Southern Hemisphere - Max Temp vs. Latitude Linear Regression
equation = linear_agression(southern["Lat"],southern["Max Temp"])
plt.title("Southern Hemisphere - Max Temp vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")
plt.savefig("PNGs/Southern Hemisphere - Max Temp vs. Latitude Linear Regression.png")

In [ ]:
#  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression
equation = linear_agression(northern["Lat"],northern["Humidity"])
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.savefig("PNGs/Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression.png")

In [ ]:
# Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression
equation = linear_agression(southern["Lat"],southern["Humidity"])
plt.title("Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.savefig("PNGs/Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression.png")

In [ ]:
# Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression
equation = linear_agression(northern["Lat"],northern["Cloudiness"])
plt.title("Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.savefig("PNGs/Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression.png")

In [ ]:
# Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression
equation = linear_agression(southern["Lat"],southern["Cloudiness"])
plt.title("Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.savefig("PNGs/Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression.png")

In [ ]:
# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression 
equation = linear_agression(northern["Lat"],northern["Wind Speed"])
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.savefig("PNGs/Northern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")

In [ ]:
# Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression
equation = linear_agression(southern["Lat"],southern["Wind Speed"])
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.savefig("PNGs/Southern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")